In [28]:
import pickle
from pathlib import Path

MODEL_PATH = Path("../models/chicago")

hotspot_models = {}
for model_file in MODEL_PATH.glob("*_statsforecast.pkl"):
    print(model_file)
    hotspot_id = model_file.stem.replace("_statsforecast","")
    print(hotspot_id)
    with open(model_file, "rb") as f:
        hotspot_models[hotspot_id] = pickle.load(f)


..\models\chicago\0.0_statsforecast.pkl
0.0
..\models\chicago\1.0_statsforecast.pkl
1.0
..\models\chicago\10.0_statsforecast.pkl
10.0
..\models\chicago\100.0_statsforecast.pkl
100.0
..\models\chicago\101.0_statsforecast.pkl
101.0
..\models\chicago\102.0_statsforecast.pkl
102.0
..\models\chicago\103.0_statsforecast.pkl
103.0
..\models\chicago\104.0_statsforecast.pkl
104.0
..\models\chicago\105.0_statsforecast.pkl
105.0
..\models\chicago\106.0_statsforecast.pkl
106.0
..\models\chicago\107.0_statsforecast.pkl
107.0
..\models\chicago\108.0_statsforecast.pkl
108.0
..\models\chicago\109.0_statsforecast.pkl
109.0
..\models\chicago\11.0_statsforecast.pkl
11.0
..\models\chicago\110.0_statsforecast.pkl
110.0
..\models\chicago\111.0_statsforecast.pkl
111.0
..\models\chicago\112.0_statsforecast.pkl
112.0
..\models\chicago\113.0_statsforecast.pkl
113.0
..\models\chicago\114.0_statsforecast.pkl
114.0
..\models\chicago\115.0_statsforecast.pkl
115.0
..\models\chicago\116.0_statsforecast.pkl
116.0
..\m

In [29]:
import pandas as pd
import numpy as np

In [30]:
DATASET_PATH = Path("../output")
df = pd.read_csv(DATASET_PATH / "violent_crimes_chicago.csv")

df["data_ocorrencia"] = pd.to_datetime(df["data_ocorrencia"])

In [36]:
forecast_list = []

for hotspot_id, model in hotspot_models.items():
    # pega os dados do hotspot específico
    hotspot_data = df[df["hotspot_id"] == float(hotspot_id)].copy()
    print(hotspot_id)
    ts = hotspot_data.groupby("data_ocorrencia").size().reset_index(name="y")
    ts["unique_id"] = str(0.0)
    ts = ts.rename(columns={"data_ocorrencia": "ds"})
    # Faz forecast
    fcst = model.forecast(df=ts.head(1000), h=14, level=[95])
    
    # Adiciona latitude/longitude do hotspot (centroide)
    lat = hotspot_data["latitude"].mean()
    lon = hotspot_data["longitude"].mean()
    fcst["latitude"] = lat
    fcst["longitude"] = lon
    fcst["hotspot_id"] = hotspot_id
    
    forecast_list.append(fcst)

# Combina tudo em um DataFrame
forecast_map = pd.concat(forecast_list, ignore_index=True)

0.0
1.0
10.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
11.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
12.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
13.0
130.0
131.0
132.0
133.0
134.0
14.0
15.0
16.0
17.0
18.0
19.0
2.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
3.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
4.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
5.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
6.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
7.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
8.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
9.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0


In [39]:
import folium

# Mapa centralizado
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=12)

# Para cada previsão, adiciona círculo e tooltip/popup
for _, row in forecast_map.iterrows():
    popup_text = (
        f"<b>Hotspot:</b> {row['hotspot_id']}<br>"
        f"<b>Data:</b> {row['ds']}<br>"
        f"<b>Crimes previstos:</b> {row['AutoARIMA']:.2f}<br>"
        f"<b>Intervalo 95%:</b> [{row['AutoARIMA-lo-95']:.2f}, {row['AutoARIMA-hi-95']:.2f}]"
    )
    
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=6 + row["AutoARIMA"],  # círculo maior para previsões maiores
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=popup_text,
        tooltip=f"{row['AutoARIMA']:.2f} crimes previstos"
    ).add_to(m)

m  # renderiza direto no notebook

In [38]:
m.save("map.html")